In [1]:
# !pip install -r requirements.txt

In [1]:
from platform import python_version

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv
load_dotenv()
import os

from typing import List, Optional
import os
from pathlib import Path

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

### альтернативный вариант

In [1]:
from platform import python_version
import os
from pathlib import Path
from typing import List, Optional

from dotenv import load_dotenv
load_dotenv()

import re
import uuid
import pandas as pd

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter

import clickhouse_connect

# (если понадобится) from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader


In [2]:
# 2.1 Настройка подключения к Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
client_qdrant = QdrantClient(url=QDRANT_URL)

# 2.2 Настройка подключения к ClickHouse
CH_HOST = '84.201.160.255'   # или из окружения
CH_PORT = 8123
CLICKHOUSE_USER = 'peter'
CLICKHOUSE_PASSWORD = '1234'

client_clickhouse = clickhouse_connect.get_client(
    host=CH_HOST,
    port=CH_PORT,
    username=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD
)


In [3]:
emails_df = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    LIMIT 100
""")


Настройка коллекции в Qdrant (с вариантами)

In [4]:
BASE_URL = "http://localhost:8000/v1"
embeddings = OpenAIEmbeddings(
    # model="Qwen/Qwen3-Embedding-8B",
    model="Qwen/Qwen3-Embedding-0.6B",
    api_key="not-needed",
    base_url=BASE_URL,
    tiktoken_enabled=False,
)

In [5]:
# тест - получаем размерность эмбединга
vec = embeddings.embed_query("test")
EMBEDDING_DIM = len(vec)
EMBEDDING_DIM

1024

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain_qdrant import QdrantVectorStore

COLLECTION_NAME = "mailkb_emails"


# ---------- Вариант A: УДАЛИТЬ коллекцию и СОЗДАТЬ ЗАНОВО ----------
def setup_collection_recreate():
    """
    Полная пересоздача коллекции: удаляем старую, создаём новую с нужной размерностью.
    Полезно, когда меняешь модель / размер эмбеддинга.
    """
    client_qdrant.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=EMBEDDING_DIM,
            distance=Distance.COSINE,
        ),
    )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если не хочешь, чтобы он дополнительно валидировал конфиг:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант B: СОЗДАТЬ, если коллекции ещё нет ----------
def setup_collection_create_if_not_exists():
    """
    Если коллекции нет — создаём.
    Если есть — НЕ трогаем (данные сохраняются).
    """
    collections = client_qdrant.get_collections().collections
    existing_names = {c.name for c in collections}

    if COLLECTION_NAME not in existing_names:
        client_qdrant.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM,
                distance=Distance.COSINE,
            ),
        )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # при желании можно отключить проверку:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант C: ИСПОЛЬЗОВАТЬ ТЕКУЩУЮ коллекцию как есть ----------
def setup_collection_use_existing():
    """
    Просто подключаемся к уже существующей коллекции.
    Ничего не создаём и не удаляем.
    Важно: размерность в Qdrant должна совпадать с размерностью эмбеддинга.
    """
    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если вдруг Qdrant создан с "левым" размером, и ты осознанно хочешь
        # отключить проверку (НЕ рекомендую в бою):
        # validate_collection_config=False,
    )
    return qv



In [7]:
# Пример использования одного из вариантов:
# qv = setup_collection_recreate()
qv = setup_collection_create_if_not_exists()
# qv = setup_collection_use_existing()

Подготовка утилит (нормализация, участники, превращение в документы)

In [8]:
RE_PREFIX = re.compile(r'^\s*(re|fw|fwd):\s*', flags=re.IGNORECASE)

def normalize_subject(subj: str) -> str:
    s = subj or ""
    while True:
        ns = RE_PREFIX.sub('', s).strip()
        if ns == s:
            break
        s = ns
    s = re.sub(r'\s+', ' ', s)
    return s.lower()

def participants_list(row) -> list[str]:
    def _norm(x):
        if not x:
            return []
        if isinstance(x, list):
            return [str(i).strip() for i in x if str(i).strip()]
        return [p.strip() for p in re.split(r'[;,]', str(x)) if p.strip()]
    people = _norm(row.get("from_addr")) + _norm(row.get("to_addr")) \
           + _norm(row.get("cc_addr")) + _norm(row.get("bcc_addr"))
    return sorted(set(people))

def build_docs(df: pd.DataFrame) -> list[Document]:
    docs: list[Document] = []
    for _, r in df.iterrows():
        subj = (r.get("subject") or "").strip()
        body = (r.get("body_text") or "").strip()
        if not body:
            body = (r.get("body_html") or "").strip()
        text = (subj + "\n\n" + body).strip()
        if not text:
            continue
        parts = participants_list(r)
        norm_subj = normalize_subject(subj)
        thread_key = f"{norm_subj}||{';'.join(sorted(parts))}"
        meta = {
            "row_id": r.get("id"),
            "message_id": r.get("message_id"),
            "subject": subj,
            "sent_at_utc": str(r.get("sent_at_utc")),
            "folder": r.get("folder"),
            "from_addr": parts[:1],
            "participants": parts,
            "thread_key": thread_key,
        }
        docs.append(Document(page_content=text, metadata=meta))
    return docs


Очистка текста и разбивка на чанки

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)

RE_QUOTED = re.compile(r"(?m)^(>+).*$")  # строки, начинающиеся с >
RE_HDR = re.compile(r"(?:^|\n)(from:|sent:|to:|subject:).*(?:\n.*){0,20}", re.IGNORECASE)

def clean_text(t: str) -> str:
    t = t.replace("\r\n", "\n")
    t = RE_HDR.sub("\n", t)
    t = RE_QUOTED.sub("", t)
    t = "\n".join([ln.strip() for ln in t.split("\n") if ln.strip()])
    return t

def preprocess_and_chunk(docs: list[Document]) -> list[Document]:
    cleaned: list[Document] = []
    for d in docs:
        txt = clean_text(d.page_content)
        if not txt:
            continue
        cleaned.append(Document(page_content=txt, metadata=d.metadata))
    return splitter.split_documents(cleaned)


Генерация стабильных ID и загрузка в Qdrant

In [10]:
def make_ids(chunks: list[Document]) -> list[str]:
    counters: dict[str, int] = {}
    ids: list[str] = []
    for c in chunks:
        msg = c.metadata.get("message_id") or c.metadata.get("row_id") or "noid"
        i = counters.get(msg, 0)
        counters[msg] = i + 1
        raw = f"{msg}::chunk_{i}"
        uid = uuid.uuid5(uuid.NAMESPACE_URL, raw)
        ids.append(str(uid))
    return ids

def process_batch(df: pd.DataFrame):
    docs = build_docs(df)
    chunks = preprocess_and_chunk(docs)
    if not chunks:
        return
    ids = make_ids(chunks)
    texts = [c.page_content for c in chunks]
    metadatas = [c.metadata for c in chunks]
    qv.add_texts(texts=texts, metadatas=metadatas, ids=ids)


Запуск загрузки (батчинг)

In [11]:
# Пример: полный цикл с батчингом
# BATCH_SIZE = 5000
# offset = 0
#
# while True:
#     df_batch = client_clickhouse.query_df(f"""
#         SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
#                sent_at_utc, folder, body_text, body_html
#         FROM mailkb.emails
#         ORDER BY sent_at_utc ASC
#         LIMIT {BATCH_SIZE} OFFSET {offset}
#     """)
#     if df_batch.empty:
#         break
#     process_batch(df_batch)
#     offset += len(df_batch)
#     print(f"Processed {offset} rows")

# Или — одноразовая загрузка небольшой выборки
offset = 0
df_small = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    LIMIT 100
""")
process_batch(df_small)
offset += len(df_small)


Аналитика переписки (треды, темы, теги проектов)

In [13]:
from collections import Counter
# Утилиты
def split_addrs(x) -> list[str]:
    if x is None:
        return []
    if isinstance(x, list):
        return [a.strip() for a in x if str(a).strip()]
    return [a.strip() for a in re.split(r'[;,]', str(x)) if a.strip()]

def participants(row) -> list[str]:
    lst = split_addrs(row.get("from_addr")) + split_addrs(row.get("to_addr")) \
        + split_addrs(row.get("cc_addr")) + split_addrs(row.get("bcc_addr"))
    return sorted(set(lst))

def clean_for_topics(text: str) -> list[str]:
    if not text:
        return []
    t = text.lower()
    t = re.sub(r'[^a-zа-я0-9\s\-]+', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    toks = t.split()
    stop = set("""
        и в во на с со от до по за для при как что это это/that of the a an to is are was were be been being
        у о об обo про из из-за над под между но или либо либо/или который которые какая какие чей чья чей-то
        re fw fwd subject тема письмо письма письме по- поводу
    """.split())
    return [w for w in toks if len(w) > 2 and w not in stop]

PROJECT_KEYWORDS = {
    "segezha": "Проект: Segezha",
    "ewm": "SAP EWM",
    "bw": "SAP BW",
    "o2c": "O2C",
    "mnf": "MNF",
    "сцбк": "СЦБК",
    "вагон": "Логистика/Вагоны",
}

def guess_project_tags(subject: str, body: str, addrs: list[str]) -> list[str]:
    text = f"{subject or ''} {body or ''}".lower()
    tags = set()
    for kw, tag in PROJECT_KEYWORDS.items():
        if kw in text:
            tags.add(tag)
    for a in addrs:
        m = re.search(r'@([a-z0-9\.-]+)', a.lower())
        if m:
            dom = m.group(1)
            if 'segezha' in dom:
                tags.add("Проект: Segezha")
            if 'bearingpoint' in dom:
                tags.add("Внутренние/Подрядчик")
    return sorted(tags)

# Добавление колонок в df
df = emails_df.copy()
df["norm_subject"] = df["subject"].apply(lambda s: normalize_subject(s if isinstance(s, str) else ""))
df["participants"] = df.apply(participants, axis=1)
df["thread_key"] = df.apply(lambda r: f'{r["norm_subject"]}||{";".join(r["participants"])}', axis=1)

def plain_body(row):
    t = (row.get("body_text") or "").strip()
    if t:
        return t
    return (row.get("body_html") or "").strip()

df["plain_body"] = df.apply(plain_body, axis=1)
df["topics"] = df.apply(lambda r: clean_for_topics(r["subject"]) + clean_for_topics(r["plain_body"][:1000]), axis=1)
df["project_tags"] = df.apply(lambda r: guess_project_tags(r["subject"], r["plain_body"], r["participants"]), axis=1)

threads = (
    df.groupby("thread_key")
      .agg(
         first_sent=("sent_at_utc","min"),
         last_sent=("sent_at_utc","max"),
         n_emails=("id","count"),
         subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),
         participants=("participants", lambda cols: sorted(set(sum(cols, [])))),
         projects=("project_tags", lambda cols: sorted(set(sum(cols, [])))),
         topics=("topics", lambda cols: [w for w,_ in Counter(sum(cols, [])).most_common(10)])
      )
      .reset_index()
      .sort_values(["last_sent","n_emails"], ascending=[False, False])
)

def flatten(col):
    out = []
    for arr in col:
        out.extend(arr)
    return out

projects_df = (
    threads.assign(project=lambda x: x["projects"].apply(lambda arr: arr if arr else ["(Не классифицировано)"]))
           .explode("project")
           .groupby("project")
           .agg(
              n_threads=("thread_key","nunique"),
              n_emails=("n_emails","sum"),
              first_sent=("first_sent","min"),
              last_sent=("last_sent","max"),
              participants=("participants", lambda cols: sorted(set(flatten(cols)))[:50]),
              top_topics=("topics", lambda cols: [w for w,_ in Counter(flatten(cols)).most_common(15)])
           )
           .reset_index()
           .sort_values(["n_threads","n_emails"], ascending=[False, False])
)


C:\Users\Peter\AppData\Local\Temp\ipykernel_81484\3129374694.py:77: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),


Сводка тредов с помощью LLM

In [42]:
from langchain_openai import ChatOpenAI
import json


llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
llm_json = ChatOpenAI(model_name="gpt-4o", temperature=0,
                      model_kwargs={"response_format": {"type": "json_object"}})

WINDOW_PROMPT = """Ты — аналитик проекта. Ниже — часть переписки (фрагмент треда).
Кратко, по делу, выпиши:
- Контекст/тема
- Ключевые действия и решения
- Сроки/дедлайны (если есть)
- Открытые вопросы/блокеры
- Фигуранты (ключевые участники, роли если видны)

Переписка:
{chunk}
"""

JSON_SCHEMA = """{
  "title": "...",
  "timeframe": { "start": "...", "end": "..." },
  "participants": ["..."],
  "stakeholders": ["..."],
  "decisions": ["..."],
  "open_items": ["..."],
  "deadlines": ["..."],
  "next_steps": ["..."],
  "project_tags": ["..."],
  "topics": ["..."]
}"""

# вариант 1
# ROLLUP_PROMPT = (
#     "Ты — аналитик проекта. На входе — список кратких сводок окон треда.\n"
#     "Собери финальную сводку переписки в формате **ЧИСТОГО JSON** без пояснений:\n"
#     "{schema}\n\n"
#     "Требования:\n"
#     "- \"title\" — 3-8 слов.\n"
#     "- списки — уникальные элементы, допускай пустые.\n"
#     "- \"project_tags\" — 1-5 ярлыков (например, \"SAP BW\", \"EWM\", \"Логистика\", \"Segezha\").\n"
#     "- \"topics\" — 3-8 ключевых тем/фраз.\n\n"
#     "Сводки окон:\n"
#     "{window_bullets}"
# )

#вариант промта 2 - оконный промт

import json  # если ещё не импортирован

WINDOW_JSON_SCHEMA = """{
  "context": "...",
  "key_points": ["..."],
  "decisions": ["..."],
  "actions": [
    {
      "owner": "...",
      "description": "...",
      "due": "..."
    }
  ],
  "open_items": ["..."],
  "deadlines": ["..."]
}"""

WINDOW_PROMPT_JSON = f"""
Ты — аналитик проекта. Ниже — фрагмент переписки (несколько писем одного треда).

Верни ИСКЛЮЧИТЕЛЬНО JSON по схеме:
{WINDOW_JSON_SCHEMA}

Правила:
- "context" — 1–3 предложения, что за задачка, без воды.
- "key_points" — важные факты, договорённости, детали.
- "decisions" — только явно принятые решения (или очень явные подразумеваемые).
- "actions" — конкретные шаги в формате "кто что делает", по возможности с датами.
- "open_items" — вопросы без ответа / неясности.
- "deadlines" — даты/периоды, когда что-то должно быть сделано.

Если информации нет — оставляй соответствующие поля пустыми массивами, ничего не выдумывай.

Переписка:
{{
chunk
}}
"""



ROLLUP_PROMPT = f"""
Ты — аналитик проекта. На входе — список кратких JSON-сводок окон треда.

Собери финальную сводку переписки в формате ЧИСТОГО JSON без пояснений, по схеме:
{JSON_SCHEMA}

Требования:
- "title" — 3–8 слов, отражает суть задачи, а не просто тему письма.
- списки — уникальные элементы, допускай пустые.
- "project_tags" — 1–5 ярлыков (например, "SAP BW", "EWM", "Логистика", "Segezha").
- "topics" — 3–8 ключевых тем/фраз.
- "next_steps" — конкретные шаги (кто что делает).

На вход тебе передаётся объект:
{{ "windows": [ ... ] }}

Где "windows" — это список JSON-объектов по схеме:
{WINDOW_JSON_SCHEMA}

Верни только один JSON-объект финального отчёта.

Входные данные (JSON):
{{"windows": windows_json}}
"""

DEFAULT_REPORT = {
    "title": "(без заголовка)",
    "timeframe": {"start": None, "end": None},
    "participants": [],
    "stakeholders": [],
    "decisions": [],
    "open_items": [],
    "deadlines": [],
    "next_steps": [],
    "project_tags": [],
    "topics": [],
}

def _coerce_report(obj: dict) -> dict:
    rep = {**DEFAULT_REPORT, **(obj or {})}
    for k in ["participants","stakeholders","decisions","open_items","deadlines","next_steps","project_tags","topics"]:
        v = rep.get(k, [])
        if not isinstance(v, list):
            v = [v] if v else []
        rep[k] = [str(x).strip() for x in v if str(x).strip()]
    tf = rep.get("timeframe") or {}
    rep["timeframe"] = {"start": tf.get("start"), "end": tf.get("end")}
    if not isinstance(rep.get("title"), str) or not rep["title"].strip():
        rep["title"] = "(без заголовка)"
    return rep

def _extract_json(text: str) -> dict:
    if not text:
        return {}
    text = text.strip()
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        try:
            return json.loads(candidate)
        except json.JSONDecodeError:
            return {}
    return {}

def _fmt_msg(row: pd.Series) -> str:
    subj = (row.get("subject") or "").strip()
    body = (row.get("plain_body") or row.get("body_text") or row.get("body_html") or "").strip()
    froms = row.get("from_addr") or []
    to = row.get("to_addr") or []
    when = str(row.get("sent_at_utc"))
    body = re.sub(r'\s+', ' ', body)
    return (
        f"=== EMAIL ===\n"
        f"Datetime: {when}\n"
        f"From: {', '.join(froms) if isinstance(froms, list) else str(froms)}\n"
        f"To: {', '.join(to) if isinstance(to, list) else str(to)}\n"
        f"Subject: {subj}\n"
        f"Body: {body[:4000]}\n"
    )

def _chunk_thread_text(thread_text: str) -> List[str]:
    splitter = TokenTextSplitter(encoding_name="cl100k_base", chunk_size=3000, chunk_overlap=300)
    return splitter.split_text(thread_text)

# def summarize_thread(thread_key: str, df: pd.DataFrame, project_candidate: Optional[str] = None) -> dict:
#     tdf = df[df["thread_key"] == thread_key].sort_values("sent_at_utc")
#     if tdf.empty:
#         rep = {**DEFAULT_REPORT, "title": "(нет писем в треде)"}
#         rep.update({"thread_key": thread_key, "n_emails": 0})
#         if project_candidate:
#             rep["project_tags"] = [project_candidate]
#         return rep
#
#     blocks = [_fmt_msg(r) for _, r in tdf.iterrows()]
#     thread_text = "\n\n".join(blocks)
#     chunks = _chunk_thread_text(thread_text)
#
#     window_summaries: List[str] = []
#     for ch in chunks:
#         resp = llm.invoke([
#             {"role": "system", "content": "Ты кратко суммируешь переписку для менеджера проекта."},
#             {"role": "user", "content": WINDOW_PROMPT.format(chunk=ch)}
#         ])
#         window_summaries.append((resp.content or "").strip())
#
#     if not window_summaries:
#         title_guess = tdf["subject"].dropna().astype(str).tail(1).values[0] if tdf["subject"].notna().any() else "(пусто)"
#         rep = {**DEFAULT_REPORT, "title": title_guess[:120]}
#         rep.update({
#             "thread_key": thread_key,
#             "n_emails": int(tdf.shape[0]),
#             "timeframe": {"start": str(tdf["sent_at_utc"].min()), "end": str(tdf["sent_at_utc"].max())},
#             "raw": "(нет сводок окон)"
#         })
#         if project_candidate:
#             rep["project_tags"] = [project_candidate]
#         return rep
#
#     joined = "\n\n---\n\n".join(window_summaries)
#     rollup_prompt_filled = ROLLUP_PROMPT.format(schema=JSON_SCHEMA, window_bullets=joined)
#
#     rollup_resp = llm_json.invoke([
#         {
#             "role": "system",
#             "content": "Ты делаешь структурированную проектную сводку по переписке. Вывод должен быть **только JSON**."
#         },
#         {"role": "user", "content": rollup_prompt_filled}
#     ]).content or ""
#
#     parsed = _extract_json(rollup_resp)
#     rep = _coerce_report(parsed)
#
#     rep["thread_key"] = thread_key
#     rep["n_emails"] = int(tdf.shape[0])
#     rep["timeframe"]["start"] = rep["timeframe"]["start"] or str(tdf["sent_at_utc"].min())
#     rep["timeframe"]["end"]   = rep["timeframe"]["end"]   or str(tdf["sent_at_utc"].max())
#
#     if project_candidate:
#         rep["project_tags"] = sorted(set(rep.get("project_tags", []) + [project_candidate]))
#
#     agg_participants = set()
#     if "participants" in tdf.columns:
#         for arr in tdf["participants"]:
#             if isinstance(arr, list):
#                 agg_participants.update(arr)
#     if agg_participants:
#         base_participants = set(rep.get("participants", []))
#         rep["participants"] = sorted(base_participants | agg_participants)
#
#     if rep.get("title") == "(без заголовка)":
#         rep["raw"] = rollup_resp
#
#     return rep



#вариант 2

def summarize_thread(thread_key: str, df: pd.DataFrame, project_candidate: Optional[str] = None) -> dict:
    tdf = df[df["thread_key"] == thread_key].sort_values("sent_at_utc")
    if tdf.empty:
        rep = {**DEFAULT_REPORT, "title": "(нет писем в треде)"}
        rep.update({"thread_key": thread_key, "n_emails": 0})
        if project_candidate:
            rep["project_tags"] = [project_candidate]
        return rep

    # 1) Собираем текст треда
    blocks = [_fmt_msg(r) for _, r in tdf.iterrows()]
    thread_text = "\n\n".join(blocks)
    chunks = _chunk_thread_text(thread_text)

    # 2) Для каждого окна делаем JSON-сводку
    window_summaries: List[dict] = []
    for ch in chunks:
        prompt = WINDOW_PROMPT_JSON.replace("chunk", ch)
        resp = llm_json.invoke([
            {"role": "system", "content": "Ты структурируешь переписку по проекту в JSON."},
            {"role": "user", "content": prompt}
        ])
        raw = resp.content or ""
        parsed = _extract_json(raw)

        if not parsed:
            # Фоллбек, если LLM что-то сломал
            parsed = {
                "context": ch[:500],
                "key_points": [],
                "decisions": [],
                "actions": [],
                "open_items": [],
                "deadlines": [],
            }

        window_summaries.append(parsed)

    # Если вообще ничего не получилось
    if not window_summaries:
        title_guess = tdf["subject"].dropna().astype(str).tail(1).values[0] if tdf["subject"].notna().any() else "(пусто)"
        rep = {**DEFAULT_REPORT, "title": title_guess[:120]}
        rep.update({
            "thread_key": thread_key,
            "n_emails": int(tdf.shape[0]),
            "timeframe": {"start": str(tdf["sent_at_utc"].min()), "end": str(tdf["sent_at_utc"].max())},
            "raw": "(нет сводок окон)"
        })
        if project_candidate:
            rep["project_tags"] = [project_candidate]
        return rep

    # 3) Rollup по JSON-окнам → финальный репорт
    windows_json_str = json.dumps(window_summaries, ensure_ascii=False)
    rollup_prompt = ROLLUP_PROMPT.replace("windows_json", windows_json_str)

    rollup_resp = llm_json.invoke([
        {"role": "system", "content": "Ты делаешь структурированную проектную сводку по переписке. Вывод должен быть ТОЛЬКО JSON."},
        {"role": "user", "content": rollup_prompt}
    ]).content or ""

    parsed = _extract_json(rollup_resp)
    rep = _coerce_report(parsed)

    # 4) Дозаполняем метаданные
    rep["thread_key"] = thread_key
    rep["n_emails"] = int(tdf.shape[0])
    rep["timeframe"]["start"] = rep["timeframe"]["start"] or str(tdf["sent_at_utc"].min())
    rep["timeframe"]["end"]   = rep["timeframe"]["end"]   or str(tdf["sent_at_utc"].max())

    if project_candidate:
        rep["project_tags"] = sorted(set(rep.get("project_tags", []) + [project_candidate]))

    # Добавим реальных участников из df, чтобы не пропали
    agg_participants = set()
    if "participants" in tdf.columns:
        for arr in tdf["participants"]:
            if isinstance(arr, list):
                agg_participants.update(arr)
    if agg_participants:
        rep["participants"] = sorted(set(rep.get("participants", []) + list(agg_participants)))

    # Если заголовок так и остался пустым — сохраним сырой ответ
    if rep.get("title") == "(без заголовка)":
        rep["raw"] = rollup_resp

    return rep


def pick_top_threads(df: pd.DataFrame, k: int = 5) -> List[str]:
    tmp = (
        df.groupby("thread_key")
          .agg(last=("sent_at_utc","max"), n=("id","count"))
          .sort_values(["last","n"], ascending=[False, False])
          .head(k)
          .reset_index()
    )
    return tmp["thread_key"].tolist()


In [43]:
print(df.head())

                                                  id  \
0                                                  1   
1                                                  1   
2                                                  1   
3  <2ec74c38-60e9-4960-9b0d-fd38216b7158@bearingp...   
4  <JIRA.31877.1614239413000.182110.1614542520003...   

                                          message_id  \
0                                            msg-001   
1                                            msg-001   
2                                            msg-001   
3  <2ec74c38-60e9-4960-9b0d-fd38216b7158@bearingp...   
4  <JIRA.31877.1614239413000.182110.1614542520003...   

                                             subject  \
0                                       Test subject   
1                                       Test subject   
2                                       Test subject   
3      Постановка оценки за 6 месяцев работы стажера   
4  SAPSP-18265 Открыть доступ к поставкам Лесо

### вывод инфо

In [44]:
projects = ["Segezha", "Норникель", "Внепроектная"]

all_reports = []

for proj in projects:
    # фильтрация
    df_proj = df[df["project_tags"].apply(lambda tags: proj in tags)]
    if df_proj.empty:
        print(f"[INFO] Нет данных для проекта '{proj}' — пропускаем.")
        continue

    thread_keys = pick_top_threads(df_proj, k=3)
    if not thread_keys:
        print(f"[INFO] Для проекта '{proj}' не найдено топ-тредов.")
        continue

    for tk in thread_keys:
        rep = summarize_thread(thread_key=tk, df=df_proj, project_candidate=proj)
        all_reports.append(rep)

# Вывод результатов
for r in all_reports:
    print(f"Project_tags: {r['project_tags']} | Title: {r['title']} | Decisions: {r.get('decisions')}")


[INFO] Нет данных для проекта 'Segezha' — пропускаем.
[INFO] Нет данных для проекта 'Норникель' — пропускаем.
[INFO] Нет данных для проекта 'Внепроектная' — пропускаем.


In [45]:
all_reports

[]

In [38]:
selected_project = "Проект: Segezha"

df_sel = df[df["project_tags"].apply(lambda tags: selected_project in tags)]
print(f"Выбран проект '{selected_project}', строк: {len(df_sel)}")

thread_keys = pick_top_threads(df_sel, k=5)
print("Найдено тредов:", thread_keys)

reports = [
    summarize_thread(thread_key=tk, df=df_sel, project_candidate=selected_project)
    for tk in thread_keys
]

for r in reports:
    print(f"Title: {r['title']} | Timeframe: {r['timeframe']} | Project_tags: {r['project_tags']} | Topics: {r['topics']}")


Выбран проект 'Проект: Segezha', строк: 75
Найдено тредов: ['sapsp-18265 открыть доступ к поставкам лесосибирского лдк||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', '[jira] (sapsp-18265) открыть доступ к поставкам лесосибирского лдк||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', 'еще раз про логистику||alexey.ivanenko@bearingpoint.ru;ce-ivan.zhilin@bearingpoint.ru;ce-valentin.konev@bearingpoint.ru;elena.zvereva@bearingpoint.ru;gorbunova_ib@segezha-group.com;kate.katushenko@bearingpoint.ru;koteshov_dv@segezha-group.com;lopatkin_va@segezha-group.com;marchik_iv@segezha-group.com;petr.ostrik@bearingpoint.ru;tatiana.egorova@bearingpoint.ru;vershinina_ea@segezha-group.com;volkova_ev@segezha-group.com', '[jira] (sapsp-17588) замечание опэ сцбк №62||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', 'sapsp-18530 не выгрузился из транспореона номер авто 6100065014||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com']
Title: Открытие доступа к поставк

In [39]:
reports

[{'title': 'Открытие доступа к поставкам Лесосибирского ЛДК',
  'timeframe': {'start': '2021-02-25 14:55:00', 'end': '2021-02-28 20:02:00'},
  'participants': ['Pustovitova_ns@segezha-group.com',
   'petr.ostrik@bearingpoint.ru',
   'sap_support@segezha-group.com',
   'Дондуков Иван',
   'Острик Пётр',
   'Шэлиговска Елена Александровна'],
  'stakeholders': [],
  'decisions': ['Запрос повторно открыт и находится в работе'],
  'open_items': ['Не определена роль в матрице мпр',
   'Неясно, настроен ли профиль'],
  'deadlines': [],
  'next_steps': ['Определить и указать роль в матрице мпр'],
  'project_tags': ['SAP', 'Segezha', 'Лесосибирский ЛДК', 'Проект: Segezha'],
  'topics': ['Открытие доступа',
   'Поставки Лесосибирского ЛДК',
   'Система SAP',
   'Роль в матрице мпр',
   'Настройка профиля'],
  'thread_key': 'sapsp-18265 открыть доступ к поставкам лесосибирского лдк||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com',
  'n_emails': 3},
 {'title': 'Открытие доступа к постав

In [47]:
selected_project = "Проект: Segezha"

df_sel = df[df["project_tags"].apply(lambda tags: selected_project in tags)]
print(f"Выбран проект '{selected_project}', строк: {len(df_sel)}")

thread_keys = pick_top_threads(df_sel, k=5)
print("Найдено тредов:", thread_keys)

reports = [
    summarize_thread(thread_key=tk, df=df_sel, project_candidate=selected_project)
    for tk in thread_keys
]

for r in reports:
    print(f"Title: {r['title']} | Timeframe: {r['timeframe']}")
    print("Decisions:", r["decisions"])
    print("Next steps:", r["next_steps"])
    print("Open items:", r["open_items"])
    print("-" * 80)


Выбран проект 'Проект: Segezha', строк: 75
Найдено тредов: ['sapsp-18265 открыть доступ к поставкам лесосибирского лдк||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', '[jira] (sapsp-18265) открыть доступ к поставкам лесосибирского лдк||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', 'еще раз про логистику||alexey.ivanenko@bearingpoint.ru;ce-ivan.zhilin@bearingpoint.ru;ce-valentin.konev@bearingpoint.ru;elena.zvereva@bearingpoint.ru;gorbunova_ib@segezha-group.com;kate.katushenko@bearingpoint.ru;koteshov_dv@segezha-group.com;lopatkin_va@segezha-group.com;marchik_iv@segezha-group.com;petr.ostrik@bearingpoint.ru;tatiana.egorova@bearingpoint.ru;vershinina_ea@segezha-group.com;volkova_ev@segezha-group.com', '[jira] (sapsp-17588) замечание опэ сцбк №62||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com', 'sapsp-18530 не выгрузился из транспореона номер авто 6100065014||petr.ostrik@bearingpoint.ru;sap_support@segezha-group.com']
Title: Настройка доступа к постав